In [231]:
!pip install dgl

In [232]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

this  segment involves directory and file operations. It creates directories, deletes directories, and extracts the contents of a ZIP file.

In [233]:
# Set the current directory as "./"
current_dir = "./"

# Set the checkpoint path by combining the current directory, the subdirectory "save_models/model_checkpoints/", and the filename "checkpoint"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"

# Create the checkpoint path directory if it doesn't exist
os.makedirs(checkpoint_path, exist_ok=True)

# Set the best model path by combining the current directory and the subdirectory "save_models/best_model/"
best_model_path = current_dir + "save_models/best_model/"

# Set the temporary data folder path by combining the current directory and the subdirectory "data_temp/"
folder_data_temp = current_dir + "data_temp/"

# Remove the temporary data folder and its contents if it exists, ignoring any errors that occur
shutil.rmtree(folder_data_temp, ignore_errors=True)

# Set the path to save as the combined path of the current directory and the filename "graph_data.zip"
path_save = current_dir + "graph_data.zip"

# Unpack the contents of the "graph_data.zip" file to the temporary data folder "folder_data_temp"
shutil.unpack_archive(path_save, folder_data_temp)


#### Custom PyTorch Datasets

the `DGLDatasetClass` class represents a classification dataset that loads DGL graphs and associated labels, masks

, and global information from a binary file. It allows for retrieving specific items from the dataset based on index.

In [234]:
""" Classification Dataset """

# Define a custom dataset class named DGLDatasetClass that extends the torch.utils.data.Dataset class.
class DGLDatasetClass(torch.utils.data.Dataset):
    # Constructor method that takes the address parameter.
    def __init__(self, address):
        # Append ".bin" to the address and store it in the self.address attribute.
        self.address = address + ".bin"

        # Load the graphs from the specified address using dgl.load_graphs() function.
        # The loaded graphs and train_labels_masks_globals dictionary are stored in self.list_graphs and train_labels_masks_globals variables, respectively.
        self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)

        # Obtain the number of graphs in the dataset by getting the length of the self.list_graphs list.
        num_graphs = len(self.list_graphs)

        # Extract the "labels", "masks", and "globals" tensors from the train_labels_masks_globals dictionary.
        # Reshape the tensors to have the shape (num_graphs, -1) and store them in self.labels, self.masks, and self.globals attributes, respectively.
        self.labels = train_labels_masks_globals["labels"].view(num_graphs, -1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs, -1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs, -1)

    # Define the __len__() method that returns the number of graphs in the dataset.
    def __len__(self):
        return len(self.list_graphs)

    # Define the __getitem__() method that returns the data at the specified index.
    def __getitem__(self, idx):
        # Return the graph, labels, masks, and globals corresponding to the given index.
        return self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]


#### Defining Train, Validation, and Test Set

the code creates instances of the `DGLDatasetClass` class for training, validation, and test sets. The `address` parameter is set differently for each set to load the corresponding dataset. Finally, the code prints the lengths of the datasets to indicate the number of graphs in each set.

In [235]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


1631 203 205


this code snippet loads the train, validation, and test datasets, concatenates them into a single dataset, and then analyzes the graph properties such as the number of vertices, edges, and graphs. It also computes and prints the shape of the adjacency matrices for each graph.

In [236]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get all the graphs in the dataset
graphs = [data[0] for data in dataset]

# Get the number of vertices, edges, and graphs
num_vertices = [graph.number_of_nodes() for graph in graphs]
num_edges = [graph.number_of_edges() for graph in graphs]
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = [graph.adjacency_matrix() for graph in graphs]
adj_shape = [matrix.shape for matrix in adj_matrix]

print("Adjacency matrix shape:", adj_shape)


Number of vertices: [8, 2, 9, 4, 1, 10, 11, 4, 4, 5, 7, 3, 2, 5, 8, 4, 6, 5, 7, 2, 4, 4, 6, 8, 4, 2, 4, 6, 5, 8, 8, 5, 10, 6, 4, 5, 5, 3, 7, 1, 3, 2, 1, 4, 6, 4, 5, 1, 1, 5, 1, 1, 8, 6, 3, 6, 9, 10, 8, 3, 7, 4, 6, 8, 6, 8, 5, 2, 5, 7, 6, 5, 4, 4, 2, 4, 5, 6, 3, 3, 4, 9, 9, 8, 10, 4, 6, 3, 4, 5, 9, 10, 4, 11, 6, 2, 12, 4, 4, 5, 2, 2, 4, 5, 8, 2, 7, 3, 5, 9, 13, 12, 5, 3, 10, 7, 10, 3, 9, 6, 4, 8, 9, 8, 4, 4, 5, 6, 5, 4, 4, 4, 3, 2, 10, 7, 7, 48, 5, 6, 8, 10, 4, 11, 7, 7, 9, 5, 11, 9, 8, 9, 10, 11, 4, 6, 11, 7, 8, 7, 5, 10, 10, 2, 9, 5, 15, 10, 5, 6, 6, 14, 11, 7, 8, 3, 8, 10, 11, 9, 7, 11, 8, 7, 8, 12, 9, 6, 9, 8, 7, 7, 5, 14, 6, 2, 8, 4, 2, 6, 11, 8, 17, 8, 12, 12, 14, 14, 12, 3, 5, 3, 7, 10, 10, 10, 7, 9, 8, 8, 8, 9, 7, 6, 10, 7, 14, 10, 8, 11, 2, 3, 4, 2, 6, 2, 3, 2, 3, 3, 4, 2, 4, 3, 3, 3, 5, 3, 5, 10, 5, 19, 21, 9, 4, 7, 3, 12, 8, 8, 9, 8, 11, 3, 5, 7, 6, 5, 5, 6, 5, 7, 16, 13, 9, 9, 4, 5, 4, 6, 6, 6, 6, 6, 17, 16, 15, 11, 10, 7, 6, 7, 11, 19, 5, 11, 5, 5, 8, 9, 5, 5, 8, 5, 14, 2, 

This  code focuses on analyzing the properties of a single graph rather than analyzing properties for all graphs in the dataset. It retrieves the first graph, calculates the number of vertices and edges, and computes the shape of the adjacency matrix.

In [237]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 8
Number of edges: 14
Number of graphs: 2039
Adjacency matrix shape: (8, 8)


In this code snippet, the cumulative counts of vertices and edges are calculated for all graphs in the dataset. Additionally, the shape of the adjacency matrix is determined

In [238]:
# Get the train, validation, and test datasets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Concatenate the datasets
dataset = torch.utils.data.ConcatDataset([train_set, val_set, test_set])

# Get the total number of graphs
num_graphs = len(dataset)

# Initialize variables to store cumulative counts
total_vertices = 0
total_edges = 0
adj_shape = None

# Iterate over all graphs in the dataset
for i in range(num_graphs):
    graph = dataset[i][0]  # Get the graph
    num_vertices = graph.number_of_nodes()  # Get the number of vertices
    num_edges = graph.number_of_edges()  # Get the number of edges

    # Update cumulative counts
    total_vertices += num_vertices
    total_edges += num_edges

    if adj_shape is None:
        adj_matrix = graph.adjacency_matrix()  # Get the adjacency matrix
        adj_shape = adj_matrix.shape  # Get the shape of the adjacency matrix

# Print the results
print("Number of vertices:", total_vertices)
print("Number of edges:", total_edges)
print("Adjacency matrix shape:", adj_shape)
print("Number of graphs:", num_graphs)


Number of vertices: 16420
Number of edges: 30238
Adjacency matrix shape: (8, 8)
Number of graphs: 2039


In this code snippet, the focus is on a single dataset rather than concatenating multiple datasets.
The code focuses on a single dataset and extracts information from the first graph in that dataset, including the number of vertices, edges, and the shape of the adjacency matrix. The results are then printed.

In [239]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_train")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)



Number of vertices: 8
Number of edges: 14
Number of graphs: 1631
Adjacency matrix shape: (8, 8)


In [240]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_val")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 11
Number of edges: 20
Number of graphs: 203
Adjacency matrix shape: (11, 11)


By loading the DGL dataset, accessing the first graph, and retrieving information such as the number of vertices, edges, and the shape of the adjacency matrix, the code provides insights into the structure of the graph stored in the dataset.

In [241]:
# Get the dataset
dataset = DGLDatasetClass(address=path_data_temp + "_test")

# Get the first graph in the dataset
graph = dataset[0][0]

# Get the number of vertices, edges, and graphs
num_vertices = graph.number_of_nodes()
num_edges = graph.number_of_edges()
num_graphs = len(dataset)

print("Number of vertices:", num_vertices)
print("Number of edges:", num_edges)
print("Number of graphs:", num_graphs)

# Get the adjacency matrix shape
adj_matrix = graph.adjacency_matrix()
adj_shape = adj_matrix.shape

print("Adjacency matrix shape:", adj_shape)


Number of vertices: 8
Number of edges: 14
Number of graphs: 205
Adjacency matrix shape: (8, 8)


The code accesses the global features from the train, validation, and test sets and prints them. Here's how it works:

1. `train_globals = [globals for _, _, _, globals in train_set]`: This line retrieves the global features from the train set and assigns them to the `train_globals` list. Each global feature corresponds to a molecule in the train set.
2. `val_globals = [globals for _, _, _, globals in val_set]`: This line retrieves the global features from the validation set and assigns them to the `val_globals` list. Each global feature corresponds to a molecule in the validation set.
3. `test_globals = [globals for _, _, _, globals in test_set]`: This line retrieves the global features from the test set and assigns them to the `test_globals` list. Each global feature corresponds to a molecule in the test set.

4. The following lines print the global features:
   - `print("Global Features for Train Set:")`: Prints a header indicating the train set global features are being displayed.
   - `for i, globals in enumerate(train_globals):`: Iterates over the train_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the train set.
   - `print()`: Prints an empty line for separation.
   - `print("Global Features for Validation Set:")`: Prints a header indicating the validation set global features are being displayed.
   - `for i, globals in enumerate(val_globals):`: Iterates over the val_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the validation set.
   - `print()`: Prints an empty line for separation.
   - `print("Global Features for Test Set:")`: Prints a header indicating the test set global features are being displayed.
   - `for i, globals in enumerate(test_globals):`: Iterates over the test_globals list.
     - `print(f"Molecule {i+1}: {globals}")`: Prints the global features for each molecule in the test set.

By accessing the global features using the tuple unpacking syntax (`for _, _, _, globals in ...`), the code retrieves and displays the global features for each molecule in the train, validation, and test sets.

In [242]:
# Access the global features
train_globals = [globals for _, _, _, globals in train_set]
val_globals = [globals for _, _, _, globals in val_set]
test_globals = [globals for _, _, _, globals in test_set]

# Print the global features
print("Global Features for Train Set:")
for i, globals in enumerate(train_globals):
    print(f"Molecule {i+1}: {globals}")
print()

print("Global Features for Validation Set:")
for i, globals in enumerate(val_globals):
    print(f"Molecule {i+1}: {globals}")
print()

print("Global Features for Test Set:")
for i, globals in enumerate(test_globals):
    print(f"Molecule {i+1}: {globals}")
print()


Streaming output truncated to the last 5000 lines.
        1.6663e-01, 4.4802e-01])
Molecule 59: tensor([2.6808e-01, 4.7353e-01, 8.0538e-01, 8.4260e-01, 8.2146e-01, 7.4952e-01,
        8.6490e-01, 8.2449e-01, 9.2169e-01, 8.8927e-01, 9.6814e-01, 9.5265e-01,
        9.7655e-01, 9.6591e-01, 9.2343e-01, 8.9984e-01, 6.8936e-07, 9.5752e-01,
        8.1364e-01, 4.6245e-01, 9.2584e-11, 5.4760e-01, 6.5828e-01, 2.2144e-01,
        3.4911e-01, 7.6285e-01, 7.2636e-01, 6.4321e-01, 6.1096e-01, 9.5991e-01,
        8.1504e-01, 7.7515e-01, 7.3211e-01, 1.0000e+00, 8.2715e-01, 6.2646e-01,
        4.8585e-01, 7.8158e-01, 7.1258e-01, 5.2736e-01, 7.1258e-01, 8.1447e-01,
        1.7704e-02, 8.1864e-01, 1.0918e-01, 4.6122e-01, 2.7139e-01, 7.4463e-01,
        7.6618e-01, 7.0728e-01, 6.9483e-01, 1.0000e+00, 1.4249e-01, 9.7600e-01,
        3.4723e-02, 4.8299e-15, 1.1178e-02, 8.2272e-01, 7.3166e-01, 4.9077e-01,
        5.6952e-08, 5.8913e-01, 9.4955e-01, 1.1424e-21, 9.6593e-01, 8.6508e-01,
        6.0286e-01, 4.5

The code accesses the output masks for the train, validation, and test sets, and then prints the masks.As you can see, there is no empty cell and values of 1 are given.

In [243]:
# Access the output masks
train_masks = train_set.masks
val_masks = val_set.masks
test_masks = test_set.masks

# Print the output masks
print("Train Set Output Masks:")
print(train_masks)
print()

print("Validation Set Output Masks:")
print(val_masks)
print()

print("Test Set Output Masks:")
print(test_masks)


Train Set Output Masks:
tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

Validation Set Output Masks:
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        

This code checks if there are any empty cells (cells with NaN values) in the train, validation, and test sets.
By iterating over the samples and checking for NaN values in the labels tensor, the code determines if there are any empty cells present in the train, validation, and test sets.

In [244]:
# Check if there is an empty cell in the train_set
empty_cells_train = any(torch.isnan(labels).any() for _, labels, _, _ in train_set)
if empty_cells_train:
    print("Train set contains empty cells.")
else:
    print("Train set does not contain empty cells.")

# Check if there is an empty cell in the val_set
empty_cells_val = any(torch.isnan(labels).any() for _, labels, _, _ in val_set)
if empty_cells_val:
    print("Validation set contains empty cells.")
else:
    print("Validation set does not contain empty cells.")

# Check if there is an empty cell in the test_set
empty_cells_test = any(torch.isnan(labels).any() for _, labels, _, _ in test_set)
if empty_cells_test:
    print("Test set contains empty cells.")
else:
    print("Test set does not contain empty cells.")


Train set does not contain empty cells.
Validation set does not contain empty cells.
Test set does not contain empty cells.


Because the data is a task, there is no empty cell

#### Data Loader

the `collate` function is used to process a batch of samples by concatenating the graphs, labels, masks, and globals. The `loader` function creates dataloaders for the training, validation, and test sets using the `DataLoader` class, with the `collate` function specified to process the batches.

In [245]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [246]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

 this code snippet iterate over the training data batch by batch, where each batch contains a graph g and other associated information (labels, masks, globals) that can be used for training model.

In [247]:
g,_,_,_=next(iter(train_dataloader))
g

Graph(num_nodes=462, num_edges=838,
      ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)})

#### Defining A GNN

##### Some Variables

 the code  sets up several parameters and configurations for training a model. These include the number of tasks, the size of global features, the number of epochs, the patience for early stopping, and the model configuration dictionary specifying the sizes of different features and hidden layers.

In [248]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


 the `GNN` class represents a Graph Neural Network (GNN) model. It takes a DGL graph and global features as input and performs graph convolutions to obtain graph-level representations. The model architecture and parameters are defined in the `__init__` method, and the forward pass is implemented in the `forward` method.

In [249]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

the `compute_score` function evaluates the model by computing the ROC AUC score for each task in a multi-task setting. It aggregates the scores and returns the average score across all tasks.

In [250]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

 the `loss_func` function calculates the loss for a multi-task binary classification problem using the binary cross-entropy loss (`BCEWithLogitsLoss`) with specified positive weights. It applies a mask to filter out padded or irrelevant elements, and the final loss is averaged over the valid elements.

In [251]:
# def loss_func(output, label, mask, num_tasks):
#     pos_weight = torch.ones((1, num_tasks))
#     pos_weight
#     criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
#     loss = mask*criterion(output,label)
#     loss = loss.sum() / mask.sum()
#     return loss

In [252]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = criterion(output, label)
    loss = loss.mean()
    return loss


#### Training and Evaluation

##### Training Function

the `train_epoch` function performs one training epoch for a graph-based model. It iterates over the batches in the training data, computes the model's predictions and the training loss, performs backpropagation to compute the gradients, updates the model's parameters, and accumulates the training loss. The function returns the average training loss per iteration for the epoch.

In [253]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

the train_evaluate function trains and evaluates a graph-based model for a specified number of epochs. It keeps track of the best validation score and saves the corresponding model checkpoint. After training, it copies the checkpoint files of the best model to a separate directory. Finally, it prints the final results, including the average validation score.

In [254]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

 the `test_evaluate` function loads the best model checkpoint, initializes a model with the same configuration, loads the model's state from the checkpoint, evaluates the model on the test dataset, and prints the test score and execution time.

In [255]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

By calling `train_evaluate()` and `test_evaluate()` one after the other, the code performs both training and testing of the graph-based model. The `start_time` variable is used to calculate and print the total execution time for both operations.

In [256]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.669 | Valid Score: 0.240
 
Epoch: 1/100 | Best Valid Score Until Now: 0.240 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.643 | Valid Score: 0.241
 
Epoch: 2/100 | Best Valid Score Until Now: 0.241 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.625 | Valid Score: 0.244
 
Epoch: 3/100 | Best Valid Score Until Now: 0.244 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.612 | Valid Score: 0.253
 
Epoch: 4/100 | Best Valid Score Until Now: 0.253 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.603 | Valid Score: 0.263
 
Epoch: 5/100 | Best Valid Score Until Now: 0.263 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.598 | Valid Score: 0.274
 
Epoch: 6/100 | Best Valid Score Until Now: 0.274 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.595 | Valid Score: 0.285
 
Epoch: 7/100 | Best Valid Score Until Now: 0.285 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.590 | Valid Score: 0.298
 
Epoch: 8/100 | Best Valid Score Until Now: 

In [257]:
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
        self.address = address + ".bin"
        self.list_graphs, train_labels_masks_globals, edge_features = dgl.load_graphs(self.address)
        num_graphs = len(self.list_graphs)
        self.labels = train_labels_masks_globals["labels"].view(num_graphs, -1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs, -1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs, -1)
        self.edge_features = edge_features.view(num_graphs, -1)

    def __len__(self):
        return len(self.list_graphs)

    def __getitem__(self, idx):
        return self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx], self.edge_features[idx]


this code demonstrates the construction of a GNN model using DGL for graph processing and message passing. The model takes a DGL graph (mol_dgl_graph) and global features (globals) as inputs and returns the aggregated node features.

In [258]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Graph convolution layers
        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

        # Global feature size
        self.global_feature_size = global_size

        # Global feature linear transformation
        self.global_transform = nn.Linear(self.global_feature_size, self.hidden_size)

        # Message function
        self.message_func = nn.Linear(self.hidden_size, self.hidden_size)

        # Reduce function
        self.reduce_func = nn.Linear(self.hidden_size * 2, self.hidden_size)

    def forward(self, mol_dgl_graph, globals):
        # Preprocess node and edge features
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # Initialize node features with initial node features
        mol_dgl_graph.ndata["h"] = mol_dgl_graph.ndata["v"]

        # Apply graph convolution to node features
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["h"])

        # Message passing
        mol_dgl_graph.update_all(
            message_func=(lambda edges: self.message_func(edges.src['h'])),
            reduce_func=(lambda nodes: torch.cat([nodes.mailbox['m'], nodes.data['h']], dim=-1))
        )

        # Get the updated node features after message passing
        h = mol_dgl_graph.ndata["h"]

        # Global feature transformation
        global_feat = self.global_transform(globals)
        global_feat = global_feat.unsqueeze(1).expand(-1, h.shape[1], -1)

        # Concatenate global feature with node features
        h = torch.cat([h, global_feat], dim=-1)

        # Apply ReLU activation
        h = F.relu(h)

        # Apply the second graph convolutional layer
        h = self.conv2(mol_dgl_graph, h)

        # Update node features with the final representation
        mol_dgl_graph.ndata["h"] = h

        # Aggregate the node features and return the mean
        return dgl.mean_nodes(mol_dgl_graph, "h")

##sage

##message copy and reduce mean

This code defines a custom SAGEConv (GraphSAGE Convolution) module in PyTorch for graph convolutional operations.
The `SAGEConv` class extends the `nn.Module` class and initializes a SAGEConv layer. It takes the input feature size (`in_feat`), output feature size (`out_feat`), and aggregator type as arguments. The aggregator type determines how neighbor information is aggregated, with `'mean'` being the default.

The `forward` method performs the graph convolution operation. It takes a DGL graph (`g`) and a tensor of node features (`h`) as inputs. The method begins by setting the node features in the graph with `g.ndata["h"] = h`. Then, it performs message passing using `update_all`. The `message_func` copies the node features to messages, while the `reduce_func` applies the specified aggregator type to aggregate the messages from neighboring nodes.
After message passing, the method retrieves the aggregated node features (`h_N`) from the graph. It concatenates the original node features (`h`) with the aggregated features (`h_N`). Finally, the concatenated features are passed through a linear transformation (`self.linear`) to obtain the output features.

In [259]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv


class SAGEConv(nn.Module):

    def __init__(self, in_feat, out_feat, aggregator_type='mean'):
        super(SAGEConv, self).__init__()
        self.aggregator_type = aggregator_type
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=getattr(fn, self.aggregator_type)("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

this code defines a GNN model that performs graph convolution operations using the SAGEConv module. It applies two SAGEConv layers to obtain node representations and aggregates them to compute the final output of the model.

In [260]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [261]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.671 | Valid Score: 0.349
 
Epoch: 1/100 | Best Valid Score Until Now: 0.349 

Patience 1
Epoch: 2/100 | Training Loss: 0.644 | Valid Score: 0.323
 
Epoch: 2/100 | Best Valid Score Until Now: 0.349 

Patience 2
Epoch: 3/100 | Training Loss: 0.621 | Valid Score: 0.322
 
Epoch: 3/100 | Best Valid Score Until Now: 0.349 

Patience 3
Epoch: 4/100 | Training Loss: 0.606 | Valid Score: 0.325
 
Epoch: 4/100 | Best Valid Score Until Now: 0.349 

Patience 4
Epoch: 5/100 | Training Loss: 0.596 | Valid Score: 0.338
 
Epoch: 5/100 | Best Valid Score Until Now: 0.349 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.592 | Valid Score: 0.359
 
Epoch: 6/100 | Best Valid Score Until Now: 0.359 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.585 | Valid Score: 0.382
 
Epoch: 7/100 | Best Valid Score Until Now: 0.382 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.581 | Valid Score: 0.411
 
Epoch: 8/100 | Best Valid Score Until Now: 0.411 

Save checkpo

##message add and reduce sum

The `SAGEConv1` class extends the `nn.Module` class and defines a SAGEConv convolutional layer. `SAGEConv1` applies a linear transformation to the concatenation of the node features and the sum of their neighbors' features.

The `forward` method of `SAGEConv1` takes a DGL graph `g` and a tensor of node features `h` as inputs. It first sets the node features in the graph `g`, then performs message passing using the `u_add_v` and `sum` built-in DGL functions to compute the sum of the neighbor features for each node. Finally, it concatenates the original node features with the sum of neighbor features, applies a linear transformation, and returns the result.

In [262]:
class SAGEConv1(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv1, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

        #The forward method of SAGEConv1 takes a DGL graph g and a tensor of node features h as inputs.
        # It first sets the node features in the graph g, then performs message passing using the u_add_v and sum built-in DGL functions to compute the sum of the neighbor features for each node.
        # Finally, it concatenates the original node features with the sum of neighbor features, applies a linear transformation, and returns the result.


    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_add_v("h","h", "m"),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


The `GNN` class extends the `nn.Module` class and initializes a GNN model. It takes a configuration dictionary (`config`), global size, and number of tasks as arguments. The model has two SAGEConv1 layers (`self.conv1` and `self.conv2`) for performing graph convolution operations.
The `forward` method performs the forward pass of the GNN model. It takes a DGL graph (`mol_dgl_graph`) and global features (`globals`) as inputs. The method first restricts the node features and edge features to their respective sizes by slicing the tensors.

Then, it applies the first SAGEConv1 layer (`self.conv1`) to the node features, followed by a ReLU activation function (`F.relu`). Next, it applies the second SAGEConv1 layer (`self.conv2`) to obtain the final node representations.

The final node representations are stored in the graph with `mol_dgl_graph.ndata["h"] = h`. Finally, the method computes the mean of the node representations using `dgl.mean_nodes` with the feature name "h" and returns the result.



In [263]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv1(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv1(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [264]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.670 | Valid Score: 0.307
 
Epoch: 1/100 | Best Valid Score Until Now: 0.307 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.621 | Valid Score: 0.344
 
Epoch: 2/100 | Best Valid Score Until Now: 0.344 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.600 | Valid Score: 0.431
 
Epoch: 3/100 | Best Valid Score Until Now: 0.431 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.583 | Valid Score: 0.518
 
Epoch: 4/100 | Best Valid Score Until Now: 0.518 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.567 | Valid Score: 0.590
 
Epoch: 5/100 | Best Valid Score Until Now: 0.590 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.548 | Valid Score: 0.669
 
Epoch: 6/100 | Best Valid Score Until Now: 0.669 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.533 | Valid Score: 0.712
 
Epoch: 7/100 | Best Valid Score Until Now: 0.712 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.518 | Valid Score: 0.739
 
Epoch: 8/100 | Best Valid Score Until Now: 

##message add and reduce sum

In [265]:
class SAGEConv2(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv2, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.send_and_recv(g.edges(), fn.u_add_v("h", "h", "m"), fn.sum("m", "h_N"))

            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv2(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv2(self.hidden_size, self.num_tasks)
#The forward method of SAGEConv2 takes a DGL graph g and a tensor of node features h as inputs.
# It first sets the node features in the graph g, then performs message passing and aggregation using the send_and_recv function.
#It sends messages by applying the "add" message function fn.u_add_v("h", "h", "m") to each edge of the graph.
#The messages are then received and aggregated using the "sum" reduce function fn.sum("m", "h_N") to compute the sum of the neighbor features for each node.
#After the message passing and aggregation step, it retrieves the aggregated neighbor features h_N from the graph data. It concatenates the original node features h with the aggregated neighbor features h_N, applies a linear transformation, and returns the result.
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.632 | Valid Score: 0.350
 
Epoch: 1/100 | Best Valid Score Until Now: 0.350 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.609 | Valid Score: 0.443
 
Epoch: 2/100 | Best Valid Score Until Now: 0.443 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.589 | Valid Score: 0.547
 
Epoch: 3/100 | Best Valid Score Until Now: 0.547 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.572 | Valid Score: 0.609
 
Epoch: 4/100 | Best Valid Score Until Now: 0.609 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.554 | Valid Score: 0.712
 
Epoch: 5/100 | Best Valid Score Until Now: 0.712 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.536 | Valid Score: 0.732
 
Epoch: 6/100 | Best Valid Score Until Now: 0.732 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.521 | Valid Score: 0.788
 
Epoch: 7/100 | Best Valid Score Until Now: 0.788 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.509 | Valid Score: 0.804
 
Epoch: 8/100 | Best Valid Score Until Now: 

##message div and reduce max

this code defines a SAGEConv3 module for graph convolutional operations, which uses "div" as the message function to compute element-wise division of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [266]:
class SAGEConv3(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv3, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_div_v("h","h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [267]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv3(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv3(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [268]:

start_time = time.time()

train_evaluate()
test_evaluate()


Patience 1
Epoch: 1/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 1/100 | Best Valid Score Until Now: 0.000 

Patience 2
Epoch: 2/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 2/100 | Best Valid Score Until Now: 0.000 

Patience 3
Epoch: 3/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 3/100 | Best Valid Score Until Now: 0.000 

Patience 4
Epoch: 4/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 4/100 | Best Valid Score Until Now: 0.000 

Patience 5
Epoch: 5/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 5/100 | Best Valid Score Until Now: 0.000 

Patience 6
Epoch: 6/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 6/100 | Best Valid Score Until Now: 0.000 

Patience 7
Epoch: 7/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 7/100 | Best Valid Score Until Now: 0.000 

Patience 8
Epoch: 8/100 | Training Loss: nan | Valid Score: 0.000
 
Epoch: 8/100 | Best Valid Score Until Now: 0.000 

Patience 9
Epoch: 9/100 | Training Loss: nan | V

##message sub and reduce mean

this code defines a SAGEConv4 module for graph convolutional operations, which uses "sub" as the message function to compute element-wise subtraction of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [269]:
class SAGEConv4(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv4, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)
#The forward method of SAGEConv4 takes a DGL graph g and a tensor of node features h as inputs. It first sets the node features in the graph g, then performs message passing and aggregation using the update_all function. It sends messages by applying the "sub" message function fn.u_sub_v("h", "h", "m") to each edge of the graph. The messages are then received and aggregated using the "mean" reduce function fn.mean("m", "h_N") to compute the mean of the neighbor features for each node.

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_sub_v('h', 'h', 'm'),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [270]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv4(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv4(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [271]:

start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.679 | Valid Score: 0.504
 
Epoch: 1/100 | Best Valid Score Until Now: 0.504 

Patience 1
Epoch: 2/100 | Training Loss: 0.660 | Valid Score: 0.388
 
Epoch: 2/100 | Best Valid Score Until Now: 0.504 

Patience 2
Epoch: 3/100 | Training Loss: 0.644 | Valid Score: 0.346
 
Epoch: 3/100 | Best Valid Score Until Now: 0.504 

Patience 3
Epoch: 4/100 | Training Loss: 0.630 | Valid Score: 0.339
 
Epoch: 4/100 | Best Valid Score Until Now: 0.504 

Patience 4
Epoch: 5/100 | Training Loss: 0.620 | Valid Score: 0.336
 
Epoch: 5/100 | Best Valid Score Until Now: 0.504 

Patience 5
Epoch: 6/100 | Training Loss: 0.608 | Valid Score: 0.348
 
Epoch: 6/100 | Best Valid Score Until Now: 0.504 

Patience 6
Epoch: 7/100 | Training Loss: 0.600 | Valid Score: 0.354
 
Epoch: 7/100 | Best Valid Score Until Now: 0.504 

Patience 7
Epoch: 8/100 | Training Loss: 0.590 | Valid Score: 0.367
 
Epoch: 8/100 | Best Valid Score Until Now: 0.504 

Patience 8
Epoch: 9/100 | T

##message mul and reduce mean

 this code defines a SAGEConv5 module for graph convolutional operations, which uses "mul" as the message function to compute element-wise multiplication of node features and "mean" as the reduce function to compute the mean of the aggregated messages from neighboring nodes.

In [272]:
class SAGEConv5(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv5, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)
#It performs message forwarding and aggregation using the ``update_all'' function. Sends messages by applying the "mul" message function "fn.u_mul_v("h", "h", "m")" to each edge of the graph. The messages are then received and summed using the mean reduction function "fn.mean("m", "h_N")" to calculate the average of the element-wise multiplied messages from the neighboring nodes.
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [273]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv5(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv5(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [274]:
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.654 | Valid Score: 0.276
 
Epoch: 1/100 | Best Valid Score Until Now: 0.276 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.636 | Valid Score: 0.283
 
Epoch: 2/100 | Best Valid Score Until Now: 0.283 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.623 | Valid Score: 0.289
 
Epoch: 3/100 | Best Valid Score Until Now: 0.289 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.614 | Valid Score: 0.295
 
Epoch: 4/100 | Best Valid Score Until Now: 0.295 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.603 | Valid Score: 0.303
 
Epoch: 5/100 | Best Valid Score Until Now: 0.303 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.595 | Valid Score: 0.311
 
Epoch: 6/100 | Best Valid Score Until Now: 0.311 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.587 | Valid Score: 0.322
 
Epoch: 7/100 | Best Valid Score Until Now: 0.322 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.583 | Valid Score: 0.336
 
Epoch: 8/100 | Best Valid Score Until Now: 

##message mul and reduce sum

 this code defines a SAGEConv6 module for graph convolutional operations, which uses "mul" as the message function to compute element-wise multiplication of node features and "sum" as the reduce function to compute the sum of the aggregated messages from neighboring nodes.

In [275]:
class SAGEConv6(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv6, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [276]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv6(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv6(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [277]:
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.719 | Valid Score: 0.639
 
Epoch: 1/100 | Best Valid Score Until Now: 0.639 

Patience 1
Epoch: 2/100 | Training Loss: 0.686 | Valid Score: 0.457
 
Epoch: 2/100 | Best Valid Score Until Now: 0.639 

Patience 2
Epoch: 3/100 | Training Loss: 0.661 | Valid Score: 0.402
 
Epoch: 3/100 | Best Valid Score Until Now: 0.639 

Patience 3
Epoch: 4/100 | Training Loss: 0.641 | Valid Score: 0.399
 
Epoch: 4/100 | Best Valid Score Until Now: 0.639 

Patience 4
Epoch: 5/100 | Training Loss: 0.618 | Valid Score: 0.487
 
Epoch: 5/100 | Best Valid Score Until Now: 0.639 

Patience 5
Epoch: 6/100 | Training Loss: 0.597 | Valid Score: 0.513
 
Epoch: 6/100 | Best Valid Score Until Now: 0.639 

Patience 6
Epoch: 7/100 | Training Loss: 0.578 | Valid Score: 0.529
 
Epoch: 7/100 | Best Valid Score Until Now: 0.639 

Patience 7
Epoch: 8/100 | Training Loss: 0.560 | Valid Score: 0.569
 
Epoch: 8/100 | Best Valid Score Until Now: 0.639 

Patience 8
Epoch: 9/100 | T

##message mul and reduce min

, the code defines a new `SAGEConv7` module for graph convolutional operations. This module uses "mul" as the message function to perform element-wise multiplication between node features, and "min" as the reduce function to compute the minimum of the aggregated messages from neighboring nodes.


In [278]:
dgl.use_libxsmm(False)  # Disable libxsmm

class SAGEConv7(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv7, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.min("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv7(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv7(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.673 | Valid Score: 0.311
 
Epoch: 1/100 | Best Valid Score Until Now: 0.311 

Patience 1
Epoch: 2/100 | Training Loss: 0.656 | Valid Score: 0.310
 
Epoch: 2/100 | Best Valid Score Until Now: 0.311 

Patience 2
Epoch: 3/100 | Training Loss: 0.640 | Valid Score: 0.310
 
Epoch: 3/100 | Best Valid Score Until Now: 0.311 

Patience 3
Epoch: 4/100 | Training Loss: 0.628 | Valid Score: 0.310
 
Epoch: 4/100 | Best Valid Score Until Now: 0.311 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.612 | Valid Score: 0.317
 
Epoch: 5/100 | Best Valid Score Until Now: 0.317 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.603 | Valid Score: 0.324
 
Epoch: 6/100 | Best Valid Score Until Now: 0.324 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.591 | Valid Score: 0.331
 
Epoch: 7/100 | Best Valid Score Until Now: 0.331 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.584 | Valid Score: 0.342
 
Epoch: 8/100 | Best Valid Score Until Now: 0.342 

Save ch

##message mul and reduce max

The `forward` method of `SAGEConv8` is similar to the previous SAGEConv layers. It sets the node features in the graph `g`, performs message passing using "mul" as the message function, and aggregation using "max" as the reduce function. It then retrieves the aggregated neighbor features `h_N` from the graph data, concatenates the original node features `h` with the aggregated neighbor features `h_N`, applies a linear transformation, and returns the result.

In [279]:
class SAGEConv8(nn.Module):

    def __init__(self, in_feat, out_feat):
        super(SAGEConv8, self).__init__()

        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func = fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.max("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [280]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv8(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv8(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [281]:
start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.687 | Valid Score: 0.358
 
Epoch: 1/100 | Best Valid Score Until Now: 0.358 

Patience 1
Epoch: 2/100 | Training Loss: 0.666 | Valid Score: 0.322
 
Epoch: 2/100 | Best Valid Score Until Now: 0.358 

Patience 2
Epoch: 3/100 | Training Loss: 0.650 | Valid Score: 0.314
 
Epoch: 3/100 | Best Valid Score Until Now: 0.358 

Patience 3
Epoch: 4/100 | Training Loss: 0.636 | Valid Score: 0.318
 
Epoch: 4/100 | Best Valid Score Until Now: 0.358 

Patience 4
Epoch: 5/100 | Training Loss: 0.622 | Valid Score: 0.325
 
Epoch: 5/100 | Best Valid Score Until Now: 0.358 

Patience 5
Epoch: 6/100 | Training Loss: 0.610 | Valid Score: 0.334
 
Epoch: 6/100 | Best Valid Score Until Now: 0.358 

Patience 6
Epoch: 7/100 | Training Loss: 0.597 | Valid Score: 0.349
 
Epoch: 7/100 | Best Valid Score Until Now: 0.358 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.588 | Valid Score: 0.367
 
Epoch: 8/100 | Best Valid Score Until Now: 0.367 

Save checkpoint
Epoch:

##add layers

`SAGEConv9`: This class extends the `nn.Module` base class from PyTorch and defines a custom graph convolutional layer using the SAGE (GraphSAGE) algorithm. It takes the input feature size `in_feat` and output feature size `out_feat` as arguments. The `forward` method performs the graph convolution operation using element-wise multiplication as the message function (`fn.u_mul_v`) and sum reduction (`fn.sum`) to aggregate messages from neighboring nodes. The resulting node features are concatenated with the original node features, passed through a linear transformation, and returned.

- `GNN`: This class extends the `nn.Module` base class and implements a Graph Neural Network model. It takes a `config` dictionary, `global_size`, and `num_tasks` as arguments. The class defines the architecture of the GNN model, which includes three instances of the `SAGEConv9` layer (`conv1`, `conv2`, `conv3`). It also includes batch normalization layers (`bn1`, `bn2`) between the convolutional layers. The `forward` method performs the forward pass of the model, applying the graph convolutional layers on the input graph `mol_dgl_graph`. It first truncates the node and edge features to the desired sizes, applies the convolutional layers with batch normalization and ReLU activation, and computes the mean of the resulting node features.
At first we work with 3 layers and in the next code we work with 4 layers

In [284]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv
import time


class SAGEConv9(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv9, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv9(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv9(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=1)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.744 | Valid Score: 0.753
 
Epoch: 1/100 | Best Valid Score Until Now: 0.753 

Patience 1
Epoch: 2/100 | Training Loss: 0.649 | Valid Score: 0.722
 
Epoch: 2/100 | Best Valid Score Until Now: 0.753 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.605 | Valid Score: 0.759
 
Epoch: 3/100 | Best Valid Score Until Now: 0.759 

Patience 1
Epoch: 4/100 | Training Loss: 0.586 | Valid Score: 0.737
 
Epoch: 4/100 | Best Valid Score Until Now: 0.759 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.554 | Valid Score: 0.792
 
Epoch: 5/100 | Best Valid Score Until Now: 0.792 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.525 | Valid Score: 0.835
 
Epoch: 6/100 | Best Valid Score Until Now: 0.835 

Patience 1
Epoch: 7/100 | Training Loss: 0.498 | Valid Score: 0.826
 
Epoch: 7/100 | Best Valid Score Until Now: 0.835 

Patience 2
Epoch: 8/100 | Training Loss: 0.481 | Valid Score: 0.812
 
Epoch: 8/100 | Best Valid Score Until Now: 0.835 

Patience 3
E

In [285]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv
import time


class SAGEConv9(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv9, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_mul_v('h', 'h', 'm'),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv9(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv9(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.conv4 = SAGEConv9(self.hidden_size, self.num_tasks)


    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=1)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.737 | Valid Score: 0.443
 
Epoch: 1/100 | Best Valid Score Until Now: 0.443 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.684 | Valid Score: 0.661
 
Epoch: 2/100 | Best Valid Score Until Now: 0.661 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.666 | Valid Score: 0.672
 
Epoch: 3/100 | Best Valid Score Until Now: 0.672 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.673 | Valid Score: 0.683
 
Epoch: 4/100 | Best Valid Score Until Now: 0.683 

Patience 1
Epoch: 5/100 | Training Loss: 0.644 | Valid Score: 0.676
 
Epoch: 5/100 | Best Valid Score Until Now: 0.683 

Patience 2
Epoch: 6/100 | Training Loss: 0.625 | Valid Score: 0.681
 
Epoch: 6/100 | Best Valid Score Until Now: 0.683 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.593 | Valid Score: 0.686
 
Epoch: 7/100 | Best Valid Score Until Now: 0.686 

Patience 1
Epoch: 8/100 | Training Loss: 0.591 | Valid Score: 0.684
 
Epoch: 8/100 | Best Valid Score Until Now: 0.686 

Patienc

the SAGEConv10 class represents a graph convolutional layer that performs message passing using element-wise addition (u_add_v) and aggregation using the maximum operation (max). The resulting features are concatenated with the original node features and transformed by a linear layer to produce the output of the layer.

In [286]:
class SAGEConv10(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv10, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_add_v('h', 'h', 'm'),
                reduce_func=fn.max("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv10(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn1
        self.conv2 = SAGEConv10(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)  # Define batch normalization layer bn2
        self.conv3 = SAGEConv10(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.conv4 = SAGEConv10(self.hidden_size, self.num_tasks)


    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

config = {}  # Replace with your config
gnn = GNN(config, num_tasks=1)  # Set num_tasks to 1 for binary classification

start_time = time.time()

train_evaluate()
test_evaluate()


Save checkpoint
Epoch: 1/100 | Training Loss: 0.599 | Valid Score: 0.659
 
Epoch: 1/100 | Best Valid Score Until Now: 0.659 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.474 | Valid Score: 0.757
 
Epoch: 2/100 | Best Valid Score Until Now: 0.757 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.433 | Valid Score: 0.785
 
Epoch: 3/100 | Best Valid Score Until Now: 0.785 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.407 | Valid Score: 0.798
 
Epoch: 4/100 | Best Valid Score Until Now: 0.798 

Patience 1
Epoch: 5/100 | Training Loss: 0.379 | Valid Score: 0.783
 
Epoch: 5/100 | Best Valid Score Until Now: 0.798 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.367 | Valid Score: 0.803
 
Epoch: 6/100 | Best Valid Score Until Now: 0.803 

Patience 1
Epoch: 7/100 | Training Loss: 0.353 | Valid Score: 0.800
 
Epoch: 7/100 | Best Valid Score Until Now: 0.803 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.335 | Valid Score: 0.804
 
Epoch: 8/100 | Best Valid Score Until Now: 0.804 

Pa